# Deutsch-Josza's Algotritm

In [16]:
from qiskit import QuantumCircuit
import numpy as np

def dj_function(num_qubits):
    """"
    Create a random DJ function
    """

    qc = QuantumCircuit(num_qubits + 1)
    if np.random.randint(0, 2):
        # Flip output qubit with 50% chance
        qc.x(num_qubits)
    if np.random.randint(0, 2):
        # Return constant circuit with a 50% chance
        return qc
    
    on_states = np.random.choice(
        range(2**num_qubits),
        2**num_qubits // 2,
        replace=False,
    )

    def add_cx(qc, bit_string):
        for qubit, bit in enumerate(reversed(bit_string)):
            if bit == "1":
                qc.x(qubit)
        return qc
    
    for state in on_states:
        qc.barrier() # Barrier to help for visuzalization
        qc = add_cx(qc, f"{state:0b}")
        qc.mcx(list(range(num_qubits)), num_qubits)
        qc = add_cx(qc, f"{state:0b}")

    qc.barrier()

    return qc

In [17]:
display(dj_function(3).draw())

q_0: ─────
          
q_1: ─────
          
q_2: ─────
     ┌───┐
q_3: ┤ X ├
     └───┘

In [18]:
def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in DJ Algorithm
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(range(n + 1))
    qc.compose(function, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

In [19]:
from qiskit_aer import AerSimulator

def dj_algorithm(function: QuantumCircuit):
    """
    Determine if DJ function is Constant or Balanced
    """
    qc = compile_circuit(function)

    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()
    if "1" in measurements[0]:
        return "balanced"
    return "constant"


In [20]:
f = dj_function(3)
display(f.draw())
display(dj_algorithm(f))

░                 ░ ┌───┐     ┌───┐ ░ ┌───┐     ┌───┐ ░           »
q_0: ──────░────────■────────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░────────■──»
           ░        │        ░ └───┘  │  └───┘ ░ ├───┤  │  ├───┤ ░ ┌───┐  │  »
q_1: ──────░────────■────────░────────■────────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──»
           ░ ┌───┐  │  ┌───┐ ░ ┌───┐  │  ┌───┐ ░ ├───┤  │  ├───┤ ░ ├───┤  │  »
q_2: ──────░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──┤ X ├─░─┤ X ├──■──»
     ┌───┐ ░ └───┘┌─┴─┐└───┘ ░ └───┘┌─┴─┐└───┘ ░ └───┘┌─┴─┐└───┘ ░ └───┘┌─┴─┐»
q_3: ┤ X ├─░──────┤ X ├──────░──────┤ X ├──────░──────┤ X ├──────░──────┤ X ├»
     └───┘ ░      └───┘      ░      └───┘      ░      └───┘      ░      └───┘»
«           ░ 
«q_0: ──────░─
«     ┌───┐ ░ 
«q_1: ┤ X ├─░─
«     ├───┤ ░ 
«q_2: ┤ X ├─░─
«     └───┘ ░ 
«q_3: ──────░─
«           ░

'balanced'

In [23]:
def bv_function(s):
    """
    Create a Bernstein-Vazirani function from a string of 1s and 0s.
    """
    qc = QuantumCircuit(len(s) + 1)
    for index, bit in enumerate(reversed(s)):
        if bit == "1":
            qc.cx(index, len(s))
    return qc

display(bv_function("1001").draw())

q_0: ──■───────
       │       
q_1: ──┼───────
       │       
q_2: ──┼───────
       │       
q_3: ──┼────■──
     ┌─┴─┐┌─┴─┐
q_4: ┤ X ├┤ X ├
     └───┘└───┘

In [24]:
def bv_algorithm(function: QuantumCircuit):
    qc = compile_circuit(function)
    result = AerSimulator().run(qc, shots=1, memory=True).result()
    return result.get_memory()[0]

display(bv_algorithm(bv_function("1001")))

'1001'